# Unit 3: Programming with RDDs

## Contents

** 3.1 Before we begin: Passing funtions to Spark **

** 3.2 Transformations **

** 3.3 Actions **

** 3.4 Loading data from HDFS **

** 3.5 Saving results back to HDFS **


## Before we begin: Passing functions to Spark

Using lambda functions:

In [7]:
rdd1 = sc.parallelize(range(4))
rdd1.collect()

[0, 1, 2, 3]

In [8]:
rdd2 = rdd1.map(lambda x: 2*x)
rdd2.collect()

[0, 2, 4, 6]

Using normal functions:

In [9]:
def double(x):
    return 2*x

In [10]:
rdd3 = rdd1.map(double)
rdd3.collect()

[0, 2, 4, 6]

Sometimes it is tricky to understand the scope and life cycle of variables and methods when running in a cluster. The main part of the code executes in the driver, but when parallel operations are done the functions passed are execute in the executors and data is passed around using **closures**.

## Transformations

### map

In [1]:
rdd1 = sc.parallelize(range(4))
rdd1.collect()

[0, 1, 2, 3]

In [2]:
rdd2 = rdd1.map(lambda x: x + 5)
rdd2.collect()

[5, 6, 7, 8]

In [3]:
def plus_five(x):
    return x + 5

rdd1.map(plus_five).collect()

[5, 6, 7, 8]

### filter

In [33]:
rdd1 = sc.parallelize(['a1', 'a2', 'b1', 'b2'])
rdd1.collect()

['a1', 'a2', 'b1', 'b2']

In [34]:
rdd2 = rdd1.filter(lambda x: 'a' in x)
rdd2.collect()

['a1', 'a2']

### flatMap

In [4]:
rdd1 = sc.parallelize(['Space: the final frontier.',
                       'These are the voyages of the starship Enterprise.'])
rdd1.collect()

['Space: the final frontier.',
 'These are the voyages of the starship Enterprise.']

In [10]:
rdd2 = rdd1.map(lambda line: line.split())
rdd2.collect()

[['Space:', 'the', 'final', 'frontier.'],
 ['These', 'are', 'the', 'voyages', 'of', 'the', 'starship', 'Enterprise.']]

In [11]:
rdd3 = rdd1.flatMap(lambda line: line.split())
rdd3.collect()

['Space:',
 'the',
 'final',
 'frontier.',
 'These',
 'are',
 'the',
 'voyages',
 'of',
 'the',
 'starship',
 'Enterprise.']

### distinct

In [12]:
rdd1 = sc.parallelize([1, 1, 1, 2, 2])
rdd1.collect()

[1, 1, 1, 2, 2]

In [13]:
rdd2 = rdd1.distinct()
rdd2.collect()

[2, 1]

## Actions

In [5]:
rdd1 = sc.parallelize([1, 1, 1, 2, 2])

### reduce

In [18]:
rdd1.reduce(lambda a, b: a + b)

7

### count

In [19]:
rdd1.count()

5

### collect

In [20]:
rdd1.collect()

[1, 1, 1, 2, 2]

### first

In [21]:
rdd1.first()

1

### take

In [22]:
rdd1.take(2)

[1, 1]

### takeSample

In [29]:
rdd1.takeSample(withReplacement=False, num=10)

[1, 1, 2, 1, 2]

In [30]:
rdd1.takeSample(withReplacement=True, num=10)

[2, 1, 2, 1, 2, 1, 1, 1, 1, 2]

## Loading data from HDFS

### textFile

In [38]:
rdd = sc.textFile('datasets/meteogalicia.txt')

In [39]:
rdd.take(5)

[u'',
 u'',
 u'ESTACI\ufffdN AUTOM\ufffdTICA:Santiago-EOAS',
 u'CONCELLO:Santiago de Compostela',
 u'PROVINCIA:A Coru\ufffda']

Several files can also be loaded together at the same time but **be careful with the number of partitions generated**:

In [31]:
rdd1 = sc.textFile('datasets/slurmd/slurmd.log.*')

In [38]:
print rdd1.toDebugString()

(10) datasets/slurmd/slurmd.log.* MapPartitionsRDD[37] at textFile at NativeMethodAccessorImpl.java:-2 []
 |   datasets/slurmd/slurmd.log.* HadoopRDD[36] at textFile at NativeMethodAccessorImpl.java:-2 []


In [35]:
rdd1.takeSample(withReplacement=False, num=5)

[u'1489952201 2017 Mar 19 20:36:41 c6604 user info slurmstepd task/cgroup: /slurm/uid_1311/job_559451/step_batch: alloc=20480MB mem.limit=20480MB memsw.limit=unlimited',
 u'1493638713 2017 May  1 13:38:33 c6606 user info slurmstepd task/cgroup: /slurm/uid_12208/job_663159/step_batch: alloc=5120MB mem.limit=5120MB memsw.limit=unlimited',
 u'1495710494 2017 May 25 13:08:14 c6601 user err slurmstepd error: gres/mic unable to set OFFLOAD_DEVICES, no device files configured',
 u'1495613369 2017 May 24 10:09:29 c6602 user info slurmstepd in _window_manager',
 u'1489747697 2017 Mar 17 11:48:17 c6605 user info slurmstepd task/cgroup: /slurm/uid_1311/job_555865: alloc=20480MB mem.limit=20480MB memsw.limit=unlimited']

### wholeTextFiles

wholeTextFiles lets you read a directory containing multiple small text files, and returns each of them as (filename, content) pairs. This is in contrast with textFile, which would return one record per line in each file.

In [36]:
rdd2 = sc.wholeTextFiles('datasets/slurmd/slurmd.log.*')

In [39]:
rdd2.toDebugString()

'(2) datasets/slurmd/slurmd.log.* MapPartitionsRDD[42] at wholeTextFiles at NativeMethodAccessorImpl.java:-2 []\n |  WholeTextFileRDD[41] at wholeTextFiles at NativeMethodAccessorImpl.java:-2 []'

In [45]:
rdd2.map(lambda (filename, content): (filename, content[:80])).collect()

[(u'hdfs://c13-19.node.int.cesga.es:8020/user/jlopez/datasets/slurmd/slurmd.log.c6601',
  u'1482336831 2016 Dec 21 17:13:51 c6601 daemon info slurmd launch task 387796.0 re'),
 (u'hdfs://c13-19.node.int.cesga.es:8020/user/jlopez/datasets/slurmd/slurmd.log.c6602',
  u'1482485639 2016 Dec 23 10:33:59 c6602 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://c13-19.node.int.cesga.es:8020/user/jlopez/datasets/slurmd/slurmd.log.c6603',
  u'1482485628 2016 Dec 23 10:33:48 c6603 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://c13-19.node.int.cesga.es:8020/user/jlopez/datasets/slurmd/slurmd.log.c6606',
  u'1482485652 2016 Dec 23 10:34:12 c6606 daemon info slurmd Slurmd shutdown complet'),
 (u'hdfs://c13-19.node.int.cesga.es:8020/user/jlopez/datasets/slurmd/slurmd.log.c6608',
  u'1482484569 2016 Dec 23 10:16:09 c6608 daemon err slurmd error: gres/mic unable t'),
 (u'hdfs://c13-19.node.int.cesga.es:8020/user/jlopez/datasets/slurmd/slurmd.log.c6604',
  u'1482485636 2016 Dec 23 10:33

### binaryRecords

In [40]:
import struct
from collections import namedtuple

AcctRecord = namedtuple('AcctRecord',
                        'flag version tty exitcode uid gid pid ppid '
                        'btime etime utime stime mem io rw minflt majflt swaps '
                        'command')

def read_record(data):
    values = struct.unpack("2BH6If8H16s", data)
    return AcctRecord(*values)

In [42]:
raw_rdd = sc.binaryRecords('datasets/pacct-20160919', recordLength=64)
records = raw_rdd.map(read_record)
records.take(2)

[AcctRecord(flag=2, version=3, tty=0, exitcode=0, uid=0, gid=0, pid=24150, ppid=24144, btime=1474162981, etime=0.0, utime=0, stime=0, mem=3924, io=0, rw=0, minflt=482, majflt=0, swaps=0, command='accton\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'),
 AcctRecord(flag=0, version=3, tty=0, exitcode=0, uid=0, gid=0, pid=24151, ppid=24144, btime=1474162981, etime=0.0, utime=0, stime=0, mem=4300, io=0, rw=0, minflt=199, majflt=0, swaps=0, command='gzip\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')]

## Saving results back to HDFS

In [ ]:
rdd.saveAsTextFile('results_directory')

It will create a separate file for each partition of the RDD.

## Exercises
Now you can try to apply the above concepts to solve the following problems:
* Unit 3 Working with meteorological data 1
* Unit 3 Calculating Pi